In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [3]:
design = designs.DesignPlanar({}, overwrite_enabled=True)
gui = MetalGUI(design)
design.chips.main.size['size_x']='8mm'
design.chips.main.size['size_y']='8mm'

In [3]:
design.chips

{'main': {'material': 'silicon',
  'layer_start': '0',
  'layer_end': '2048',
  'size': {'center_x': '0.0mm',
   'center_y': '0.0mm',
   'center_z': '0.0mm',
   'size_x': '8mm',
   'size_y': '8mm',
   'size_z': '-750um',
   'sample_holder_top': '890um',
   'sample_holder_bottom': '1650um'}}}

In [4]:

cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))


# q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
#                                             pos_y='-2000um',
#                                             chip='main',
#                                             connection_pads=cp0))]
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
                                            pos_y='-2000um',
                                            chip='main'))]
                                            
qubits=q0
q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = LeftCoupler(design, "c0", options=dict(pos_x=f"{q0x-150}um",
                                              pos_y=f"{q0y+140}um",
                                              pad_width="150um",
                                              pad_height="30um",
                                              cpw_stub_height="200um",
                                              chip="main"))
design.rebuild()
gui.rebuild()
gui.autoscale()

In [5]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="main",
                                         pos_x = '-3000um',
                                         pos_y = '2000um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="main",
                                         pos_x = '3000um',
                                         pos_y = '2000um',
                                         orientation='180',
                                         lead_length = '30um'))                                        

In [6]:
clt0 = CoupledLineTee(design, "clt0", 
                        options=dict(pos_x='-2000um',
                                    pos_y='2000um', 
                                    coupling_length='300um',
                                    chip="main",
                                    down_length='300um',
                                    fillet='99um',
                                    coupling_space='5um',
                                    mirror=True,
                                    open_termination=True))

In [7]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="main", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_start'))))
fl1=RouteStraight(design, 'fl1',options=Dict(chip="main",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [8]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="main",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

# Resonator length

In [9]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [10]:
resonator_length_2=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,2)
    a=a.strip('mm')
    a=float(a)-0.35
    resonator_length_2.append(a)
print(resonator_length_2)


[8.103254683870285, 7.874788339841421, 7.658346540209131]


In [11]:
rr0=connect('rr0', 'clt0', 'second_end', 'c0', 'in',
            resonator_length_2[1],jogs,jogs, 
            '400um','0um', '100um', '100um')
            

# EPR Analysis

In [12]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [13]:
hfss.start()

INFO 04:44PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:44PM [load_ansys_project]: 	Opened Ansys App
INFO 04:44PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 04:44PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project26
INFO 04:44PM [connect_design]: No active design found (or error getting active design).
INFO 04:44PM [connect]: 	 Connected to project "Project26". No design detected


True

In [14]:
hfss.activate_ansys_design("Planar_chip", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','clt0','rr0','fl0','fl1'],
                    [('clt0','prime_end')],
                    box_plus_buffer=False)

04:44PM 58s WARNING [activate_ansys_design]: The design_name=Planar_chip was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 04:44PM [connect_design]: 	Opened active design
	Design:    Planar_chip [Solution type: Eigenmode]
WARNING 04:44PM [connect_setup]: 	No design setup detected.
WARNING 04:44PM [connect_setup]: 	Creating eigenmode default setup.
INFO 04:44PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [22]:
hfss.close()


True

# LOM Analysis

In [15]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [16]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.85',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.85',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [19]:
c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 06:57PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
INFO 06:57PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 06:57PM [analyze]: Analyzing setup Setup
INFO 06:58PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpdf96jzvq.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5850000000, Maxwell, 1, False
INFO 06:58PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpsuy_vpel.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5850000000, Maxwell, 1, False
INFO 06:58PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp0dgwg_h1.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5850000000, Maxwell, 2, False
INFO 06:58PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpiayxuicj.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 58500000

,ground_main_plane,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground_main_plane,184.370,-47.1480,-40.209,-37.5130
pad_bot_Q0,-47.148,87.7230,-34.511,-2.4963
pad_top_Q0,-40.209,-34.5110,102.250,-24.3570
quad_coupler_c0,-37.513,-2.4963,-24.357,65.4790


In [20]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 6.80
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.654075 [GHz]
EC 299.959286 [MHz]
EJ 14.854145 [GHz]
alpha -344.808994 [MHz]
dispersion 12.822727 [KHz]
Lq 10.995564 [nH]
Cq 64.576191 [fF]
T1 19.710973 [us]

**Coupling Properties**

tCqbus1 -9.176144 [fF]
gbus1_in_MHz -137.523686 [MHz]
χ_bus1 -7.721457 [MHz]
1/T1bus1 8074.433555 [Hz]
T1bus1 19.710973 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,6.059985,347.776841,14.854145,-405.380119,50.327602,[-108.35747322551042],[-11.290340396472738],11.290340,108.357473
2,6.049443,346.483485,14.854145,-403.722046,48.686526,[-117.28744369849758],[-12.890429887118596],12.890430,117.287444
3,5.972246,337.097184,14.854145,-391.722064,38.054481,[-115.9738844128741],[-10.505510068816047],10.505510,115.973884
4,5.869859,324.875838,14.854145,-376.18456,27.168573,[-126.56590545096087],[-9.99630096093328],9.996301,126.565905
5,5.775448,313.834591,14.854145,-362.231371,19.699009,[-129.49803785552487],[-8.629758938580627],8.629759,129.498038
6,5.73745,309.452232,14.854145,-356.715231,17.255639,[-133.11549596485824],[-8.465707856296902],8.465708,133.115496
7,5.692485,304.311683,14.854145,-350.260617,14.718509,[-134.4821537777365],[-7.93064381887283],7.930644,134.482154
8,5.673428,302.147727,14.854145,-347.548609,13.748487,[-136.7760198113813],[-7.916180796931697],7.916181,136.776020
9,5.654075,299.959286,14.854145,-344.808994,12.822727,[-137.52368551810528],[-7.721457133344024],7.721457,137.523686


In [1]:
c1.sim.close()

NameError: name 'c1' is not defined